# Metrics Heatmap

<a href="https://colab.research.google.com/github/netdata/netdata-community/blob/main/netdata-agent-api/netdata-pandas/metrics_heatmap.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will use the [netdata-pandas](https://github.com/netdata/netdata-pandas) Python package to pull some data from some demo Netdata servers and make some pretty looking heatmaps, because we all love a good heatmap don't we. 

**Note**: you can click the "Open in Colab" button above to open this notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true) where you can just get going with it without having to set up python enviornments or any messy stuff like that.

In [13]:
# if you need to, uncomment below to install netdata-pandas and seaborn packages
#!pip install plotly==4.14.1 netdata-pandas==0.0.33

In [14]:
import numpy as np
import pandas as pd
from netdata_pandas.data import get_data
import plotly.express as px

Lets pull some data for the last 15 minutes.

In [15]:
# inputs
hosts = ['london.my-netdata.io']
charts_regex = 'system.*'
before = 0
after = -60*15
resample_freq = '10s'

In [16]:
# get the data
df_raw = get_data(hosts=hosts, charts_regex=charts_regex, after=after, before=before, index_as_datetime=True)
print(df_raw.shape)
df_raw.head()

(901, 72)


,system.active_processes|active,system.cpu|guest,system.cpu|guest_nice,system.cpu|iowait,system.cpu|irq,system.cpu|nice,system.cpu|softirq,system.cpu|steal,system.cpu|system,system.cpu|user,...,system.softirqs|RCU,system.softirqs|SCHED,system.softirqs|TASKLET,system.softirqs|TIMER,system.softnet_stat|dropped,system.softnet_stat|flow_limit_count,system.softnet_stat|processed,system.softnet_stat|received_rps,system.softnet_stat|squeezed,system.uptime|uptime
time_idx,,,,,,,,,,,,,,,,,,,,,
2020-12-17 20:51:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-17 20:51:21,210.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.750000,1.500000,0.500000,...,126.14915,94.65477,23.01293,136.30220,0.0,0.0,63.98060,0.0,0.0,67374290.0
2020-12-17 20:51:22,210.0,0.0,0.0,0.0,0.0,0.0,0.248139,0.496278,1.736973,1.488834,...,138.79278,119.52900,35.99115,164.43971,0.0,0.0,95.92684,0.0,0.0,67374290.0
2020-12-17 20:51:23,210.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.750000,1.000000,1.250000,...,134.15971,112.09761,27.97567,156.49800,0.0,0.0,76.00372,0.0,0.0,67374290.0
2020-12-17 20:51:24,210.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.249377,0.997506,1.246883,...,110.56150,100.66286,18.98655,134.26354,0.0,0.0,58.00659,0.0,0.0,67374290.0


## Heatmaps!

In [17]:
# lets resample to 5 sec frequency
df = df_raw.resample(resample_freq).mean()

# lets min-max normalize our data so metrics can be compared on a heatmap
df=(df-df.min())/(df.max()-df.min())

# drop na cols
df = df.dropna(how='all', axis=1)

# lets sort cols by their std to try make heatmap prettier
df = df[df.std().sort_values(ascending=False).index]

print(df.shape)
df.head(10)

(91, 47)


,system.ram|free,system.ram|cached,system.ipv6|received,system.ram|used,system.uptime|uptime,system.load|load5,system.entropy|entropy,system.load|load15,system.load|load1,system.processes|blocked,...,system.ctxt|switches,system.interrupts|MCP,system.cpu|user,system.idlejitter|min,system.interrupts|virtio2-req.0_32,system.ram|buffers,system.softnet_stat|squeezed,system.cpu|softirq,system.ipv6|sent,system.active_processes|active
time_idx,,,,,,,,,,,,,,,,,,,,,
2020-12-17 20:51:20,0.835711,0.000000,0.827305,0.330601,0.000000,0.225301,0.190847,0.837546,0.000000,1.0,...,0.487620,0.0,0.234727,0.322185,0.349767,1.0,0.0,0.054866,1.0,0.159420
2020-12-17 20:51:30,0.789123,0.004010,0.469320,0.378248,0.010112,0.182280,0.220837,0.692308,0.000000,1.0,...,0.369704,0.0,0.236175,0.397993,0.342714,0.0,0.0,0.149499,1.0,0.000000
2020-12-17 20:51:40,0.724509,0.006888,0.062605,0.445031,0.021348,0.182280,0.253821,0.692308,0.000000,1.0,...,0.552596,0.0,0.113157,0.227425,0.227420,0.0,0.0,0.199877,1.0,0.304348
2020-12-17 20:51:50,0.667849,0.022711,0.050355,0.499937,0.032584,0.136710,0.288013,0.692308,0.000000,1.0,...,0.421200,0.0,0.355817,0.498328,0.685496,0.0,0.0,0.197045,1.0,0.347826
2020-12-17 20:52:00,0.684889,0.024804,0.619428,0.481534,0.043820,0.136710,0.319791,0.692308,0.000000,1.0,...,0.271508,0.0,0.208932,0.705188,0.428435,0.0,0.0,0.195823,1.0,0.304348
2020-12-17 20:52:10,0.684355,0.029250,0.752282,0.480854,0.055056,0.136710,0.351971,0.692308,0.000000,1.0,...,0.404967,0.0,0.257725,0.515050,0.342748,0.0,0.0,0.148024,1.0,0.304348
2020-12-17 20:52:20,0.614933,0.054163,0.775954,0.546671,0.066292,0.136710,0.384151,0.538462,0.000000,1.0,...,0.398462,0.0,0.169620,0.481605,0.685496,0.0,0.0,0.147558,1.0,0.869565
2020-12-17 20:52:30,0.514500,0.062860,0.934969,0.649330,0.077528,0.091140,0.417136,0.538462,0.000000,1.0,...,0.420734,0.0,0.135786,0.461538,0.342748,0.0,0.0,0.148017,1.0,1.000000
2020-12-17 20:52:40,0.695845,0.059198,0.272585,0.460680,0.088764,0.096256,0.450121,0.538462,0.005307,1.0,...,0.511396,0.0,0.140722,0.528428,0.342748,0.0,0.0,0.099252,1.0,0.304348


In [20]:
from plotly.offline import iplot

fig = px.imshow(df.transpose())
fig.update_layout(
    autosize=False,
    width=1000,
    height=1200)
iplot(fig)